In [13]:
import csv
import re
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download NLTK stopwords if not already downloaded
import nltk
nltk.download('stopwords')
nltk.download('punkt')

import csv

articles_raw = []

with open('sampled.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        articles_raw.append(row)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bilal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bilal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
import re
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download NLTK stopwords if not already downloaded
import nltk
nltk.download('stopwords')
nltk.download('punkt')

# Function to preprocess text
def preprocess_text(text):
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = text.lower()  # Convert to lowercase
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    return filtered_text

# Create dictionary to store processed articles
articles_dict = defaultdict(dict)

for article in articles_raw:
    article_id = int(article['ARTICLE_ID'])
    title = article['TITLE']
    section_title = article['SECTION_TITLE']
    section_text = article['SECTION_TEXT']
    
    # Preprocess text
    processed_text = preprocess_text(section_text)
    
    # Store processed article in dictionary
    articles_dict[article_id] = {
        'title': title,
        'section_title': section_title,
        'section_text': processed_text
    }


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bilal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bilal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:
import re
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download NLTK stopwords if not already downloaded
import nltk
nltk.download('stopwords')
nltk.download('punkt')

# Function to preprocess text
def preprocess_text(text):
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = text.lower()  # Convert to lowercase
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    return filtered_text

# Create dictionary to store processed articles
articles_dict = defaultdict(dict)
articles_dict

for article in articles_raw:
    article_id = int(article['ARTICLE_ID'])
    title = article['TITLE']
    section_title = article['SECTION_TITLE']
    section_text = article['SECTION_TEXT']
    
    # Preprocess text
    processed_text = preprocess_text(section_text)
    processed_text
    
    # Store processed article in dictionary
    articles_dict[article_id] = {
        'title': title,
        'section_title': section_title,
        'section_text': processed_text
    }


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bilal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bilal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [16]:
def display_articles(articles_dict):
    for article_id, article_data in articles_dict.items():
        print(f"Article ID: {article_id}")
        print(f"Title: {article_data['title']}")
        print(f"Section Title: {article_data['section_title']}")
        print(f"Section Text: {article_data['section_text']}\n")

# Display articles in dictionary
display_articles(articles_dict)


Article ID: 55627
Title: Delavan Illinois
Section Title: History
Section Text: ['delavan', 'founded', 'group', 'settlers', 'new', 'england', 'city', 'derives', 'name', 'edward', 'c', 'delavan', 'temperance', 'advocate', 'albany', 'new', 'york', 'post', 'office', 'operation', 'delavan', 'since', '1840']

Article ID: 63340
Title: Silver Creek Township Lake County Minnesota
Section Title: Introduction
Section Text: ['silver', 'creek', 'township', 'township', 'lake', 'county', 'minnesota', 'united', 'states', 'population', '1178', '2000', 'census', 'minnesota', 'state', 'highway', '61', 'serves', 'main', 'route', 'township', 'silver', 'creek', 'township', 'organized', '1905']

Article ID: 69485
Title: Holbrook New York
Section Title: References
Section Text: []

Article ID: 84731
Title: Violent Femmes (album)
Section Title: External links
Section Text: ['violent', 'femmes', 'adobe', 'flash', 'radio3net', 'streamed', 'copy', 'licensed']

Article ID: 98327
Title: Kragerø
Section Title: Histo

In [17]:
def generate_candidates(frequent_itemsets, k):
    candidates = []
    for itemset1 in frequent_itemsets:
        for itemset2 in frequent_itemsets:
            if itemset1 != itemset2:
                candidate = list(set(itemset1) | set(itemset2))
                candidate.sort()
                if len(candidate) == k and candidate not in candidates:
                    candidates.append(candidate)
    return candidates

def calculate_support(articles_dict, candidates):
    support_count = {}
    for candidate in candidates:
        for article_id, article_data in articles_dict.items():
            article_words = set(article_data['section_text'])
            if set(candidate).issubset(article_words):
                support_count.setdefault(tuple(candidate), 0)
                support_count[tuple(candidate)] += 1
    total_articles = len(articles_dict)
    support = {candidate: count / total_articles for candidate, count in support_count.items()}
    return support

def prune_candidates(support, min_support):
    frequent_itemsets = []
    for candidate, support_value in support.items():
        if support_value >= min_support:
            frequent_itemsets.append(list(candidate))
    return frequent_itemsets

def apriori(articles_dict, min_support):
    frequent_itemsets = []
    k = 1
    # Generate frequent itemsets of size 1
    candidates = [[word] for article_id in articles_dict for word in articles_dict[article_id]['section_text']]
    support = calculate_support(articles_dict, candidates)
    frequent_itemsets.extend(prune_candidates(support, min_support))
    
    # Generate frequent itemsets of size > 1
    while True:
        k += 1
        candidates = generate_candidates(frequent_itemsets, k)
        if not candidates:
            break
        support = calculate_support(articles_dict, candidates)
        frequent_itemsets.extend(prune_candidates(support, min_support))
    
    return frequent_itemsets

# Call apriori function
min_support = 0.1  # Example support threshold
frequent_itemsets = apriori(articles_dict, min_support)

# Save articles after Apriori
# Implement saving mechanism here


In [18]:
def save_articles_after_apriori(articles_dict, frequent_itemsets):
    filtered_articles = {}
    for article_id, article_data in articles_dict.items():
        article_words = set(article_data['section_text'])
        for itemset in frequent_itemsets:
            if set(itemset).issubset(article_words):
                filtered_articles[article_id] = article_data
                break  # If an article contains at least one frequent itemset, add it and move to the next article
    return filtered_articles

# Call the function to save articles after Apriori
filtered_articles = save_articles_after_apriori(articles_dict, frequent_itemsets)

# Print the filtered articles
for article_id, article_data in filtered_articles.items():
    print(f"Article ID: {article_id}, Title: {article_data['title']}, Section Title: {article_data['section_title']}, Section Text: {article_data['section_text']}")


Article ID: 55627, Title: Delavan Illinois, Section Title: History, Section Text: ['delavan', 'founded', 'group', 'settlers', 'new', 'england', 'city', 'derives', 'name', 'edward', 'c', 'delavan', 'temperance', 'advocate', 'albany', 'new', 'york', 'post', 'office', 'operation', 'delavan', 'since', '1840']
Article ID: 63340, Title: Silver Creek Township Lake County Minnesota, Section Title: Introduction, Section Text: ['silver', 'creek', 'township', 'township', 'lake', 'county', 'minnesota', 'united', 'states', 'population', '1178', '2000', 'census', 'minnesota', 'state', 'highway', '61', 'serves', 'main', 'route', 'township', 'silver', 'creek', 'township', 'organized', '1905']
Article ID: 98327, Title: Kragerø, Section Title: History, Section Text: ['town', 'kragerø', 'established', 'municipality', '1', 'january', '1838', 'see', 'formannskapsdistrikt', 'days', 'sailing', 'ships', 'kragerø', 'one', 'norways', 'largest', 'port', 'cities', 'rural', 'municipalities', 'sannidal', 'skåtøy', 

In [19]:
import random

# Select a random article from the dictionary
random_article_id = random.choice(list(articles_dict.keys()))

# Take user input
user_query = input("Enter your query: ")


In [20]:
def jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0

# Find relevant articles based on user input and random article
query_words = set(user_query.split())
random_article_words = set(articles_dict[random_article_id]['section_text'])

similar_articles = []
for article_id, article_data in articles_dict.items():
    article_words = set(article_data['section_text'])
    similarity = jaccard_similarity(query_words, article_words)
    if similarity >= 0.5:
        similar_articles.append((article_id, article_data['title'], similarity))

# Print similar articles
print("Articles similar to user query:")
for article_id, title, similarity in similar_articles:
    print(f"Article ID: {article_id}, Title: {title}, Similarity: {similarity}")

# Display articles with high similarity
def display_high_similarity_articles(similar_articles):
    sorted_similar_articles = sorted(similar_articles, key=lambda x: x[2], reverse=True)
    print("\nArticles with high similarity:")
    for article_id, title, similarity in sorted_similar_articles[:5]:  # Display top 5 articles
        print(f"Article ID: {article_id}, Title: {title}, Similarity: {similarity}")

# Call the function to display articles with high similarity
display_high_similarity_articles(similar_articles)


Articles similar to user query:

Articles with high similarity:


In [23]:
import random

# Select a random article from the dictionary
random_article_id = random.choice(list(articles_dict.keys()))

# Take user input
user_query = input("Enter your query: ")

# Find relevant articles based on user input and random article
query_words = set(preprocess_text(user_query))
random_article_words = set(articles_dict[random_article_id]['section_text'])

similar_articles = []
for article_id, article_data in articles_dict.items():
    article_words = set(article_data['section_text'])
    similarity = jaccard_similarity(query_words, article_words)
    similar_articles.append((article_id, article_data['title'], similarity))

# Print similar articles
print("Articles similar to user query:")
for article_id, title, similarity in similar_articles:
    print(f"Article ID: {article_id}, Title: {title}, Similarity: {similarity}")

# Display articles with high similarity
def display_high_similarity_articles(similar_articles):
    sorted_similar_articles = sorted(similar_articles, key=lambda x: x[2], reverse=True)
    print("\nArticles with high similarity:")
    for article_id, title, similarity in sorted_similar_articles[:5]:  # Display top 5 articles
        print(f"Article ID: {article_id}, Title: {title}, Similarity: {similarity}")

# Call the function to display articles with high similarity
display_high_similarity_articles(similar_articles)


Articles similar to user query:
Article ID: 55627, Title: Delavan Illinois, Similarity: 0.0
Article ID: 63340, Title: Silver Creek Township Lake County Minnesota, Similarity: 0.0
Article ID: 69485, Title: Holbrook New York, Similarity: 0.0
Article ID: 84731, Title: Violent Femmes (album), Similarity: 0.0
Article ID: 98327, Title: Kragerø, Similarity: 0.0
Article ID: 117812, Title: Brainstem, Similarity: 0.0
Article ID: 130726, Title: Julian Byng 1st Viscount Byng of Vimy, Similarity: 0.0
Article ID: 140586, Title: Gina Lynn, Similarity: 0.0
Article ID: 162554, Title: Dorpat Voivodeship, Similarity: 0.0
Article ID: 215054, Title: Oberon-class submarine, Similarity: 0.0043859649122807015
Article ID: 249055, Title: Jessye Norman, Similarity: 0.0
Article ID: 251652, Title: Khmelnytskyi Ukraine, Similarity: 0.0
Article ID: 270848, Title: Pavel Kubina, Similarity: 0.0
Article ID: 279986, Title: Internal ballistics, Similarity: 0.0
Article ID: 299479, Title: Swimming at the 2004 Summer Olympi